# 1. 나만의 DBMS 만들어보기

* 우리는 지난 스터디를 하면서 나만의 DBMS를 만들정도의 실력들을 쌓았습니다.
* 데이터타입, 파일스트림, 객체지향 프로그래밍, 그리고 SQL까지 재료는 다 갖춰져있습니다.
* 안믿긴다구요? 한번 해봅시다.

## Learning Objective:
* 지난 시간 배운것들에 대한 복습 (이 프로그램을 우리가 절차지향 방식으로 만든다고 생각해 봅시다.)
* 데이터 전처리 및 정규화가 중요한 이유
* compiler
* 내가 생각하는 기능이 추가되면 추가될 수록 코드는 기하급수적으로 늘어난다
* 그렇기 때문에 DDD (Domain Driven Design) 혹은 OOP의 개념의 태동
* 설계의 중요성
* 때때로 다른사람이 짠 코드를 분석하여 내 방식대로 짜야 할 때가 있다.
* 어떨 때엔 처음부터 다시 코드를 짜는것이 더 빠른 방법일 수 있다...! (개발에 있어서 자주 모든 코드가 갈아 엎어지는 이유)

In [635]:
import os, sys, time

In [636]:
class queryParser:
    
    def __init__(self, query):
        self.dql = ['select']
        self.dml = ['insert', 'update', 'delete']
        self.ddl = ['create', 'alter', 'drop']
        
        self.command_set = {}
        # 우리 DBMS의 한계!
        self.q = query.lower().strip().split()
        
        self.command_set['mode'] = self.q[0]
        self.command_set['target'] = self.q[1]
        self.command_set['name'] = self.q[2]
        self.command_set['task'] = {}
        self.command_set['options'] = {}
    
    
    def generate_job_command(self):
        if self.q[0] in self.dql:
            self.parse_dql()
            return self.command_set

        # DML 질의 요청시
        elif self.q[0] in self.dml:
            self.parse_dml()
            return self.command_set
        
        
        # DDL 질의 요청시
        elif self.q[0] in self.ddl:
            self.parse_ddl()
            return self.command_set
        
        else:
            raise Exception('SYNTAX ERROR')
    
    
    

    def parse_dql(self):
        target = query.split('from')[0].split('select')[-1].replace(' ', '').split(',')
        
        if len(target) == 1:
            self.command_set['target'] = target[0]
        else:
            self.command_set['target'] = target
        
        self.command_set['name'] = self.q[3]
        
        option_set = query.split('from')[-1].split()[1:]
        
        condition = option_set[0]
        sub = option_set[1]
        operand = option_set[2]
        obj = option_set[3]
        
        self.command_set['options']['condition'] = condition
        self.command_set['options']['subject'] = sub 
        self.command_set['options']['operand'] = operand
        self.command_set['options']['object'] = obj
    
    
    
    
    def parse_dml(self):
        # 메타데이터 파싱
        if self.command_set['mode'] == 'insert':
            
            # 선택한 컬럼값들을 변경할때 -> 모든 컬럼값이 아니라 하나임으로
            col = self.q[2]
            if '(' in col and ')' in col:
                insert_keys= col.split('(')[1].split(')')[0].split(',')
                
                # 테이블 이름 재지정
                self.command_set['name'] = col.split('(')[0]
                for insert_key in insert_keys:
                    self.command_set['task'].setdefault('column', []).append(insert_key)
                    
                    
            values = self.q[-1].split('(')[1].split(')')[0].split(',')
            for value in values:
                self.command_set['task'].setdefault('values',[]).append(value)
            
                
        
        elif self.command_set['mode'] == 'update':
            pass
        
        elif self.command_set['mode'] == 'delete':
            pass
        
        else:
            raise Exception('SYNTAX ERROR')

        
    def parse_ddl(self):
        # DLL 질의에서 create 요청시
        if self.command_set['mode'] == 'create':

            # create 문에서 작성한 메타데이터 파싱
            commands = ' '.join(self.q[4:-1]).split(',')
            for command in commands:
                command = command.lstrip().split()

                self.command_set['task'].setdefault('column', []).append(command[0])
                self.command_set['task'].setdefault('type', []).append(command[1])

                
        
        # DDL 질의에서 change 요청시
        elif self.command_set['mode'] == 'alter':
            
            # alter 문에서 작성한 메타데이터 파싱
            command = ' '.join(self.q[3:]).split(' ')            
            self.command_set['task']['job'] = command[0]
            
            # alter 는 add, modify, change, drop이 있다!
            # 편의상 change만 작성하자.
            if self.command_set['task']['job'] == 'add':
                self.command_set['task']['meta_target'] = command[1]
                self.command_set['task']['target_name'] = command[2]
                self.command_set['task']['target_type'] = command[3]

            elif self.command_set['task']['job'] == 'change':
                self.command_set['task']['meta_target'] = command[1]
                self.command_set['task']['target_name'] = command[2]
                self.command_set['task']['target_new_name'] = command[3]
                if len(command) == 7:
                    self.command_set['task']['target_new_type'] = command[4]
                else:
                    self.command_set['task']['target_new_type'] = None
                    

            elif self.command_set['task']['job'] == 'drop':
                self.command_set['task']['meta_target'] = command[1]
                self.command_set['task']['target_name'] = command[2]

            else:
                raise Exception('SYNTAX ERROR')
        
        
        # 직접 만들어 보세요!
        elif self.command_set['mode'] == 'drop':
            print('this is dropping query')             
        
        
        else:
            raise Exception('SYNTAX ERROR')

In [637]:
# query = 'select * from XXX where id = 1'

# query = 'insert into XXX(1,2,3) values(hjo,dhon,dc)'
# query = 'insert into XXX values(dc,oc)'

# query = '''

#     create table XXX (
#         id int,
#         name varchar(20),
#         age int
#     )
    
# '''

# query = 'alter table xxx add column sex varchar(255)'
# query = 'alter table xxx change column sex sex2 varchar(112)'
# query = 'alter table xxx drop column sex'

# query = 'drop table xxx'

qp = queryParser(query)
print(qp.generate_job_command())

{'mode': 'insert', 'target': 'into', 'name': 'members', 'task': {'values': ['1', '2', '3', '4']}, 'options': {}}


In [633]:
import os
import ast
class ourDatabase:
    def __init__(self):
        pass
    
    def command_executor(self, command):
        
        mode = command['mode']
        target = command['target']
        name = command['name']
        task = command['task']
        options = command['options']
        
        print(mode)
        if mode == 'create':
            self.create(**command)
        
        elif mode == 'alter':
            self.alter(**command)    
            
        elif mode == 'insert':
            self.insert(**command)
            
        elif mode == 'update':
            pass
        
        elif mode == 'delete':
            pass
            
        elif mode == 'select':
            pass
        
            
        else:
            raise Exception('SYNTAX ERROR')
            

    
    def create(self, **kwargs):
#         if os.path.isfile(kwargs['name']):
#             raise Exception('Duplicated Table!')

        with open(kwargs['name'], 'w') as f:
            print(kwargs['task']['column'])
            f.write(str(kwargs['task']['column']))
            
        with open(f"{kwargs['name']}.meta", 'w') as mf:
            print(kwargs['task']['type'])
            mf.write(str(kwargs['task']['type']))
            

        print(f'Table {kwargs["name"]} created')
        
        
    def alter(self, **kwargs):
        if not os.path.isfile(kwargs['name']):
            raise Exception('Duplicated Table!')
        
        if kwargs['task']['job'] == 'add':
            with open(kwargs['name'], 'r') as f:
                data = f.readline()
                data = ast.literal_eval(data)
                data.append(kwargs['task']['target_name'])

            with open(f'{kwargs["name"]}.meta', 'r') as mf:
                metadata = mf.readline()
                metadata = ast.literal_eval(metadata)
                metadata.append(kwargs['task']['target_type'])

            with open(kwargs['name'], 'w') as f:
                f.write(str(data))

            with open(f'{kwargs["name"]}.meta', 'w') as mf:
                mf.write(str(metadata))
        
            print(f'Table {kwargs["name"]} altered')
            
            
            
        elif kwargs['task']['job'] == 'change':
            pass
        
        
        elif kwargs['task']['job'] == 'drop':
            pass
        
        else:
            raise Exception('SYNTAX ERROR')
    
    
    def drop(self, **kwargs):
        pass
    
    
    
    def insert(self, **kwargs):
        print(kwargs)
        if not os.path.isfile(kwargs['name']):
            raise Exception('No such table')
        
        values = kwargs['task']['values']
        
        with open(kwargs['name'], 'r') as f:
            data = f.readline()
            keys = ast.literal_eval(data)
        
        with open(f'{kwargs["name"]}.meta', 'r') as mf:
            metadata = mf.readline()
            metadata = ast.literal_eval(metadata)
        
        if not len(values) == len(keys):
            raise Exception('Value objects mismatching')
            
        for value, metadatum in zip()

In [638]:
# query = '''

#     create table members (
#         id int,
#         name varchar(10),
#         age int
#     )
    
# '''

# query = 'alter table members add column sex varchar(10)'

query = 'insert into members values(1,2,3,4)'

db = ourDatabase()
qp = queryParser(query)
db.command_executor(qp.generate_job_command())

insert
{'mode': 'insert', 'target': 'into', 'name': 'members', 'task': {'values': ['1', '2', '3', '4']}, 'options': {}}
['int', 'varchar(10)', 'int', 'varchar(10)']


In [202]:
def query_parser(query):
    # 입력 받을 커맨드 정의
    dql = ['select']
    dml = ['insert', 'update', 'delete']
    ddl = ['create', 'alter', 'drop']
    
    # 입력 쿼리 정규화
    q = query.lower().strip().split()
    
    # 쿼리 기본 문법 확인: 어떤 질의인지 확인
    selector = q[0]
    
    # 명령셋 합성
    command_set = {}
    
    
    if selector in dql:
        pass
    
    
    elif selector in dml:
        command_set['mode'] = q[0]
        command_set['target'] = q[1]
        command_set['name'] = q[2]
    
    
    elif selector in ddl:
        command_set['mode'] = q[0]
        command_set['target'] = q[1]
        command_set['name'] = q[2]
        
        
        # DDL 질의에서 create 요청시
        if selector == 'create':
            
            # create 문에서 작성한 메타데이터 파싱
            commands = ' '.join(q[4:-1]).split(',')
            for command in commands:
                command = command.lstrip().split()
                
                command_set.setdefault('column', []).append(command[0])
                command_set.setdefault('type', []).append(command[1])
            
        # DDL 질의에서 alter 요청시
        elif selector == 'alter':
            
    
            # alter 문에서 작성한 메타데이터 파싱
            command = ' '.join(q[3:]).split(' ')            
            command_set['job'] = command[0]
            
            # alter 는 add, modify, change, drop이 있다!
            # 편의상 change 만 작성하자
            if command_set['job'] == 'add':
                command_set['job_target'] = command[1]
                command_set['target_name'] = command[2]
                command_set['target_type'] = command[3]
            
            elif command_set['job'] == 'change':
                command_set['job_target'] = command[1]
                command_set['target_name'] = command[2]
                command_set['target_new_name'] = command[3]
                command_set['target_type'] = command[4]
                
            
            elif command_set['job'] == 'drop':
                command_set['job_target'] = command[1]
                command_set['target_name'] = command[2]
                
            else:
                raise Exception('SYNTAX ERROR')
                
        else:
            # 직접 만들어 보세요!
            print('this is dropping query')
    
    
    else:
        raise Exception('SYNTAX ERROR')
    
    
    
    return command_set

In [203]:
query = 'insert into XXX(id, name) values(1,2)'

query = 'insert into XXX values(1,2,3)'

In [204]:
query_parser(query)

{'mode': 'alter',
 'target': 'table',
 'name': 'xxx',
 'job': 'drop',
 'job_target': 'column',
 'target_name': 'abc'}

### 1.1.2. ALTER

In [113]:
def alter_database():
    pass

### 1.1.3. Drop

In [79]:
def drop_database(path):
    if not os.path.isfile(path):
        raise Exception(f'No Such Database: {path}')
    os.remove(path)
    print(f'Database {path} Deleted')

In [198]:
drop_database(p)

Database db.bin Deleted


## 1.2. DML 만들기

### 1.2.1. Insert

In [230]:
def insert(path, data):
    if not os.path.isfile(path):
        raise Exception(f'No Such Database: {path}')
    
    # 두번째...
    with open(path,'r') as rf:
        idx = rf.readlines()
    idx = len(idx) + 1
    
    with open(path, 'a') as f:
        # d = [1, data] # 첫번째..
        d = (idx, data)
        f.write(str(d) + '\n')
        print(f'{data} inserted')

In [231]:
insert(p, 'yes!')

yes! inserted


In [232]:
def select(path):
    import ast
    if not os.path.isfile(path):
        raise Exception(f'No Such Database: {path}')
        
    with open(path, 'r') as f:
        data = f.readlines()
        
    # 두번째...
    trimmed = []
    for datum in data:    
        datum = datum.replace('\n', '')
        datum = ast.literal_eval(datum)
        trimmed.append(datum)
    
    return trimmed

In [233]:
select(p)

[[1, 'yes!'],
 [2, 'yes!'],
 [3, 'yes!'],
 (4, 'yes!'),
 (5, 'yes!'),
 (6, 'yes!'),
 (7, 'yes!')]